In [2]:
!pip install -qU llama-index llama-index-llms-ollama llama-index-embeddings-huggingface qdrant-client llama-index-vector-stores-qdrant

In [4]:
import nest_asyncio
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader, StorageContext
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import Settings
import qdrant_client

/home/mb/anaconda3/envs/py_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-15 06:48:09.446538: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734245289.508590    6557 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734245289.520799    6557 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 06:48:09.624225: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critic

In [5]:
# allows nested access to the event loop
nest_asyncio.apply()

In [6]:
# add your documents in this directory, you can drag & drop
input_dir_path = './docs'

In [7]:
collection_name="chat_with_docs"

client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

def create_index(documents):
    vector_store = QdrantVectorStore(client=client, collection_name=collection_name)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
    )
    return index

In [8]:
# setup llm & embedding model and reranker
llm=Ollama(model="llama3.2", request_timeout=120.0)
embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)
rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=3
)

/home/mb/anaconda3/envs/py_env/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
# load data
loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".pdf"],
            recursive=True
        )
docs = loader.load_data()

# Creating an index over loaded data
Settings.embed_model = embed_model
try:
    index = create_index(docs)
    print('Using Qdrant collection')
except:
    index = VectorStoreIndex.from_documents(docs, show_progress=True)

# Create the query engine, where we use a cohere reranker on the fetched nodes
Settings.llm = llm
query_engine = index.as_query_engine(
    similarity_top_k=10, node_postprocessors=[rerank]
)

# ====== Customise prompt template ======
qa_prompt_tmpl_str = (
"Context information is below.\n"
"---------------------\n"
"{context_str}\n"
"---------------------\n"
"Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
"Query: {query_str}\n"
"Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# Generate the response
response = query_engine.query("What is the difference between Artificial intelligence, machine learning and deep learning?",)

Using Qdrant collection


In [10]:
display(Markdown(str(response)))

To answer your question, I'll break it down step by step:

1. **Artificial Intelligence (AI)**: AI refers to the broader field of research and development aimed at creating machines that can perform tasks typically required of humans. It encompasses various techniques, including machine learning, rule-based systems, expert systems, and more.

2. **Machine Learning (ML)**: Machine learning is a subset of artificial intelligence that focuses on developing algorithms and statistical models that enable machines to learn from data, without being explicitly programmed.

3. **Deep Learning (DL)**: Deep learning is a subfield of machine learning that involves the use of neural networks with multiple layers to analyze and interpret data. These networks are inspired by the structure and function of the human brain.

Now, summarizing the differences:

- **Artificial Intelligence** encompasses all AI-related techniques and domains.
- **Machine Learning** is a subset of AI focused on enabling machines to learn from data without being explicitly programmed.
- **Deep Learning** is a subfield of machine learning that uses neural networks with multiple layers to analyze complex data.

So, in summary: AI includes ML and DL as subsets, while ML is a broader field than DL.